<a href="https://colab.research.google.com/github/mshinohar/langchain-book/blob/main/langchain_ch3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ChatGPTをAPIから利用するために

## 3-3 入出力の長さの制限や課金に影響する「トークン」

### Tokenizerとtiktokenの紹介

In [ ]:
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 10.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.


In [ ]:
import tiktoken

text = "LLMを使ってクールなものを作るのは簡単だが、プロダクションで使えるものを作るのは非常に難しい。"

encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")
tokens = encoding.encode(text)
print(len(tokens))

49


### 日本語のトークン数について

In [ ]:
text = "It’s easy to make something cool with LLMs, but very hard to make something production-ready with them."

encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")
tokens = encoding.encode(text)
print(len(tokens))

23


## 3-4 Chat Completions APIにふれる環境準備

### Google Colabのノートブック作成

In [ ]:
print("Hello World")

### OpenAIのAPIキーの準備

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "your api key"

## 3-5 Chat Completions APIをさわってみる

### OpenAIのライブラリ

In [ ]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 2.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.


### Chat Completions APIの呼び出し

In [ ]:
import openai

response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Hello! I'm John."}
  ]
)

print(response)

{
  "id": "chatcmpl-8DS3Btm4aX3NJwTPmvoIFiuCEyxNC",
  "object": "chat.completion",
  "created": 1698216781,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Hello John! How can I assist you today?"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 23,
    "completion_tokens": 10,
    "total_tokens": 33
  }
}


### 会話履歴を踏まえた応答を得る

In [ ]:
openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Hello! I'm John."},
        {"role": "assistant", "content": "Hello John! How can I assist you today?"},
        {"role": "user", "content": "Do you know my name?"}
    ]
)

<OpenAIObject chat.completion id=chatcmpl-8DS4DPLBqORiERPse6Y0jQo96ArX7 at 0x7b460e8178d0> JSON: {
  "id": "chatcmpl-8DS4DPLBqORiERPse6Y0jQo96ArX7",
  "object": "chat.completion",
  "created": 1698216845,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Yes, you mentioned your name is John. How can I assist you further, John?"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 47,
    "completion_tokens": 18,
    "total_tokens": 65
  }
}

### ストリーミングで応答を得る

In [ ]:
response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Hello! I'm John."}
  ],
  stream=True
)

for chunk in response:
  choice = chunk["choices"][0]
  if choice["finish_reason"] is None:
    print(choice["delta"]["content"])


Hello
 John
!
 How
 can
 I
 assist
 you
 today
?


### （コラム）Completions API

In [ ]:
import openai

response = openai.Completion.create(
  model="text-davinci-003",
  prompt="Hello! I'm John."
)

print(response)

{
  "warning": "This model version is deprecated. Migrate before January 4, 2024 to avoid disruption of service. Learn more https://platform.openai.com/docs/deprecations",
  "id": "cmpl-8DS5cPHA1zFTnvJ0XnuFreca3ILFq",
  "object": "text_completion",
  "created": 1698216932,
  "model": "text-davinci-003",
  "choices": [
    {
      "text": "\n\nNice to meet you John! My name is Mary.",
      "index": 0,
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 6,
    "completion_tokens": 13,
    "total_tokens": 19
  }
}


In [ ]:
prompt = """Human: Hello! I'm John.
AI: Nice to meet you, John!
Human: Do you know my name?
AI: """

response = openai.Completion.create(
  model="text-davinci-003",
  prompt=prompt
)
print(response)

{
  "warning": "This model version is deprecated. Migrate before January 4, 2024 to avoid disruption of service. Learn more https://platform.openai.com/docs/deprecations",
  "id": "cmpl-8DS9ZWoIJpbeS41xF5N2UppmfBgIt",
  "object": "text_completion",
  "created": 1698217177,
  "model": "text-davinci-003",
  "choices": [
    {
      "text": " No, I don't know your name.",
      "index": 0,
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 31,
    "completion_tokens": 9,
    "total_tokens": 40
  }
}


## 3-6 Function calling

### Function callingのサンプルコード

[OpenAI の公式ドキュメント](https://platform.openai.com/docs/guides/gpt/function-calling) をもとに一部改変したコードです。

In [ ]:
import json

def get_current_weather(location, unit="celsius"):
    weather_info = {
        "location": location,
        "temperature": "25",
        "unit": "celsius",
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)

In [ ]:
functions = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. Tokyo",
                },
                "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
            },
            "required": ["location"],
        },
    }
]

In [ ]:
messages = [{"role": "user", "content": "What's the weather like in Tokyo?"}]

response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions
)

print(response)

{
  "id": "chatcmpl-8DSCnWGKONFZOBMYWPjsh2XTVXnTF",
  "object": "chat.completion",
  "created": 1698217377,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "get_current_weather",
          "arguments": "{\n  \"location\": \"Tokyo\",\n  \"unit\": \"celsius\"\n}"
        }
      },
      "finish_reason": "function_call"
    }
  ],
  "usage": {
    "prompt_tokens": 79,
    "completion_tokens": 25,
    "total_tokens": 104
  }
}


In [ ]:
response_message = response["choices"][0]["message"]

available_functions = {
    "get_current_weather": get_current_weather,
}
function_name = response_message["function_call"]["name"]
fuction_to_call = available_functions[function_name]
function_args = json.loads(response_message["function_call"]["arguments"])

function_response = fuction_to_call(
    location=function_args.get("location"),
    unit=function_args.get("unit"),
)

print(function_response)

{"location": "Tokyo", "temperature": "25", "unit": "celsius", "forecast": ["sunny", "windy"]}


In [ ]:
messages.append(response_message)
messages.append(
    {
        "role": "function",
        "name": function_name,
        "content": function_response,
    }
)

In [ ]:
second_response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=messages,
)

print(second_response)

{
  "id": "chatcmpl-8DSFDe3nmBYZpUMkxIatKKMJHGwmA",
  "object": "chat.completion",
  "created": 1698217527,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "The current weather in Tokyo is sunny and windy with a temperature of 25\u00b0C."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 80,
    "completion_tokens": 17,
    "total_tokens": 97
  }
}
